In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import itertools
from collections import defaultdict

# Importing Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import img_to_array,  load_img

from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator 
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model,Sequential


## Second model
Using a pre-trained model ontop of our own

## 1. Choose a Pretrained Model:
Selection: Use VGG16 from tensorflow.keras.applications for transfer learning.

In [2]:
from tensorflow.keras.applications import VGG16

model_VGG16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

## 2. Preprocess the Images:
Resize and Convert to Tensor:
Resize the images to the required input shape (32x32x3) and convert them to tensors. Convert Grayscale to RG - Resize and Normalize.

In [3]:
# data generator that will load and preprocess your grayscale images, convert them to RGB, 
# resize them to the required input shape (32x32x3), and normalize the pixel values to the range [0, 1]. 
# This data can be fed to the VGG16 model for transfer learning.


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    samplewise_center=False,
    samplewise_std_normalization=False,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    validation_split=0.2,  # Split for validation
    dtype='float32'
)

# Load the images and convert to tensors
train_generator = train_datagen.flow_from_directory(
    'content/train',
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',  # Convert grayscale to RGB
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'content/train',
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',  # Convert grayscale to RGB
    subset='validation'
)

Found 23060 images belonging to 7 classes.
Found 5761 images belonging to 7 classes.


## 3. Setup the Transfer Learning Model:
Extract Features with Pretrained Base Model:
Remove the top layers of the VGG16 model and use only the bottom layers.
Freeze the weights of the pretrained layers to prevent further training.

In [4]:
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras.models import load_model

# Load your emotion classification model
emotion_model = load_model('model.h5', compile=False)

# Modify the emotion model to accept the VGG16 output shape (48, 48, 1)
emotion_model_input = Input(shape=(48, 48, 1))
emotion_model_output = emotion_model(emotion_model_input)

# Flatten the VGG16 output
vgg16_output = Flatten()(model_VGG16.output)

# Concatenate the outputs
combined_output = Concatenate()([vgg16_output, emotion_model_output])

# Add more layers as needed for your specific task
# For example:
x = Dense(128, activation='relu')(combined_output)
output = Dense(7, activation='softmax')(x)  # Assuming 7 classes for your emotion model

# Create the combined model
combined_model = Model(inputs=[model_VGG16.input, emotion_model_input], outputs=output)

# Freeze the base VGG16 layers
for layer in model_VGG16.layers:
    layer.trainable = False

# Compile the combined model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
combined_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 32, 32, 64)           1792      ['input_1[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 32, 32, 64)           36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)           0         ['block1_conv2[0][0]']        
                                                                                              

### Limit the Dataset Size

In [5]:
# Combine batches from the limited generators
from itertools import islice
from skimage.transform import resize

def custom_train_generator(generator, emotion_model):
    target_size = (48, 48, 1)  # Set the desired target size for emotion model
    while True:
        for batch_data, batch_labels in generator:
            # Resize the data to match emotion model's input shape
            resized_batch_data = np.array([resize(img, target_size) for img in batch_data])
            emotion_predictions = emotion_model.predict(resized_batch_data)
            yield [resized_batch_data, emotion_predictions], batch_labels

def custom_validation_generator(generator, emotion_model):
    target_size = (48, 48, 1)  # Set the desired target size for emotion model
    while True:
        for batch_data, batch_labels in generator:
            # Resize the data to match emotion model's input shape
            resized_batch_data = np.array([resize(img, target_size) for img in batch_data])
            emotion_predictions = emotion_model.predict(resized_batch_data)
            yield [resized_batch_data, emotion_predictions], batch_labels


In [11]:
from itertools import islice

def limit_generator(generator, limit):
    for i, (data, labels) in enumerate(generator):
        if i >= limit:
            break
        yield data, labels

# Define the desired dataset size
desired_dataset_size = 500

# Calculate the number of batches to take based on the available batches
train_batches_to_take = min(desired_dataset_size // train_generator.batch_size, len(train_generator))
validation_batches_to_take = min(desired_dataset_size // validation_generator.batch_size, len(validation_generator))

### Testing data

In [12]:
print("Number of batches in the training generator:", len(train_generator))
print("Number of batches in the validation generator:", len(validation_generator))
print("Number of samples per batch in the training generator:", train_generator.batch_size)
print("Number of samples per batch in the validation generator:", validation_generator.batch_size)


Number of batches in the training generator: 721
Number of batches in the validation generator: 181
Number of samples per batch in the training generator: 32
Number of samples per batch in the validation generator: 32


In [13]:
for i, (data_batch, label_batch) in enumerate(train_generator):
    print("Batch", i)
    print("Data shape:", data_batch.shape)
    print("Label shape:", label_batch.shape)
    if i == 1:  # Print the first 5 batches
        break

Batch 0
Data shape: (32, 32, 32, 3)
Label shape: (32, 7)
Batch 1
Data shape: (32, 32, 32, 3)
Label shape: (32, 7)


### Custom Data Generator

In [14]:
# Reset the generators
limited_train_generator = islice(train_generator, train_batches_to_take)
limited_validation_generator = islice(validation_generator, validation_batches_to_take)

# Create modified generators
limited_train_generator = custom_train_generator(limited_train_generator, emotion_model)
limited_validation_generator = custom_validation_generator(limited_validation_generator, emotion_model)

### Fit the Model

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reset the generators
limited_train_generator = islice(train_generator, train_batches_to_take)
limited_validation_generator = islice(validation_generator, validation_batches_to_take)

# Create modified generators
limited_train_generator = custom_train_generator(limited_train_generator, combined_model)
limited_validation_generator = custom_validation_generator(limited_validation_generator, combined_model)

# Train the model
history = combined_model.fit(
    limited_train_generator,
    epochs=30,
    validation_data=limited_validation_generator,
    callbacks=[early_stopping]
)


ValueError: in user code:

    File "C:\Users\asche\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\asche\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\asche\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\asche\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\asche\AppData\Roaming\Python\Python39\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\asche\AppData\Roaming\Python\Python39\s

In [ ]:
# Evaluate the model on the validation dataset
loss, accuracy = combined_model.evaluate(limited_validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)